In [4]:
1 + (2 == 3)


1

In [9]:
from itertools import pairwise
from math import inf
from typing import List


class Solution:

    def boxDelivering(self, boxes: List[List[int]], portsCount: int, maxBoxes: int, maxWeight: int) -> int:
        # dp[i] 前i个箱子运到对应码头的行程数
        # dp[i] = min(dp[j]+cost(j,i-1)) # 前j个子问题+最后一趟将[j,i-1]的箱子运到对应码头的行程数
        # i-j<=maxBoxes w(j,i-1)<=maxWeight cost(j,i-1))=[j,i-1]相邻码头不等的个数+过去和返回的两趟
        # dp[0] = 0
        n = len(boxes)
        cost = [int(x[0] != y[0]) for x, y in pairwise(boxes)]  # cost[i] i和i+1是否去一个码头
        pre_weight = [0] * (n + 1)  # [0,i-1]的总重量 [i,j]:p[j+1]-p[i]
        pre_cost = [0] * n  # [0,i]的相邻不等码头总个数 [i,j]:p[j]-p[i]
        for i in range(n):
            pre_weight[i + 1] = pre_weight[i] + boxes[i][1]
        for i in range(n - 1):
            pre_cost[i + 1] = pre_cost[i] + cost[i]
        dp = [inf] * (n + 1)
        dp[0] = 0
        for i in range(1, n + 1):
            for j in range(max(0, i - maxBoxes), i):
                if pre_weight[i] - pre_weight[j] > maxWeight:
                    continue
                dp[i] = min(dp[i], dp[j] + pre_cost[i - 1] - pre_cost[j] + 2) # 优化点:只与dp[j] - pre_cost[j] 最小值相关
        return dp[-1]


s = Solution()
s.boxDelivering([[1, 1], [2, 1], [1, 1]], 2, 3, 3)


4

In [3]:
from itertools import pairwise
from math import inf
from typing import List
from heapq import heappop, heappush
# 堆优化

class Solution:

    def boxDelivering(self, boxes: List[List[int]], portsCount: int, maxBoxes: int, maxWeight: int) -> int:
        # dp[i] 前i个箱子运到对应码头的行程数
        # dp[i] = min(dp[j]+cost(j,i-1)) # 前j个子问题+最后一趟将[j,i-1]的箱子运到对应码头的行程数
        # i-j<=maxBoxes w(j,i-1)<=maxWeight cost(j,i-1))=[j,i-1]相邻码头不等的个数+过去和返回的两趟
        # dp[0] = 0
        n = len(boxes)
        cost = [int(x[0] != y[0]) for x, y in pairwise(boxes)]  # cost[i] i和i+1是否去一个码头
        pre_weight = [0] * (n + 1)  # [0,i-1]的总重量 [i,j]:p[j+1]-p[i]
        pre_cost = [0] * n  # [0,i]的相邻不等码头总个数 [i,j]:p[j]-p[i]
        for i in range(n):
            pre_weight[i + 1] = pre_weight[i] + boxes[i][1]
        for i in range(n - 1):
            pre_cost[i + 1] = pre_cost[i] + cost[i]
        dp = [inf] * (n + 1)
        dp[0] = 0
        pq = [(0, 0)]  # (dp[0]-pre_cost[0],i=0)
        for i in range(1, n + 1):
            # 优化点:只与dp[j] - pre_cost[j] 最小值相关
            while pq and (pq[0][1] < max(0, i - maxBoxes) or pre_weight[i] - pre_weight[pq[0][1]] > maxWeight):
                heappop(pq)
            dp[i] = pq[0][0] + pre_cost[i - 1] + 2
            if i < n:
                heappush(pq, (dp[i] - pre_cost[i], i))
        return dp[-1]


s = Solution()
s.boxDelivering([[1, 1], [2, 1], [1, 1]], 2, 3, 3)


4

In [5]:
from itertools import pairwise
from math import inf
from typing import List
from collections import deque
# 最小队列优化


class Solution:

    def boxDelivering(self, boxes: List[List[int]], portsCount: int, maxBoxes: int, maxWeight: int) -> int:
        # dp[i] 前i个箱子运到对应码头的行程数
        # dp[i] = min(dp[j]+cost(j,i-1)) # 前j个子问题+最后一趟将[j,i-1]的箱子运到对应码头的行程数
        # i-j<=maxBoxes w(j,i-1)<=maxWeight cost(j,i-1))=[j,i-1]相邻码头不等的个数+过去和返回的两趟
        # dp[0] = 0
        n = len(boxes)
        cost = [int(x[0] != y[0]) for x, y in pairwise(boxes)]  # cost[i] i和i+1是否去一个码头
        pre_weight = [0] * (n + 1)  # [0,i-1]的总重量 [i,j]:p[j+1]-p[i]
        pre_cost = [0] * n  # [0,i]的相邻不等码头总个数 [i,j]:p[j]-p[i]
        for i in range(n):
            pre_weight[i + 1] = pre_weight[i] + boxes[i][1]
        for i in range(n - 1):
            pre_cost[i + 1] = pre_cost[i] + cost[i]
        dp = [inf] * (n + 1)
        dp[0] = 0
        q = deque([(0, 0)])  # (dp[0]-pre_cost[0],i=0)
        for i in range(1, n + 1):
            # 优化点:只与dp[j] - pre_cost[j] 最小值相关
            while q and (q[0][1] < max(0, i - maxBoxes) or pre_weight[i] - pre_weight[q[0][1]] > maxWeight):
                q.popleft()
            dp[i] = q[0][0] + pre_cost[i - 1] + 2
            if i < n:
                cur = dp[i] - pre_cost[i]
                while q and cur < q[-1][0]:  # 维护一个递增队列 队头就是最小值
                    q.pop()
                q.append((cur, i))
        return dp[-1]


s = Solution()
s.boxDelivering([[1, 1], [2, 1], [1, 1]], 2, 3, 3)


4

In [ ]:
'a'.is